In [8]:
import weaviate
from sentence_transformers import SentenceTransformer
from weaviate_filter import GraphQLFilter


In [5]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [2]:
WEAVIATE_URL = ""
WEAVIATE_API_KEY = ""
client = weaviate.Client(
    url=WEAVIATE_URL,  # Replace w/ your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key=WEAVIATE_API_KEY),  # Replace w/ your Weaviate instance API key
)

In [28]:
genders = ["M", "F"]
countries = ["Germany"]
labels = ["dlf", "building"]


filter = GraphQLFilter()
filter.add_list_conditions_as_operands(["gender"], "Equal", "valueText", genders, "Or")
filter.add_list_conditions_as_operands(["birth_country"], "Equal", "valueText", countries, "Or")
filter.add_conditions_for_multiple_paths(labels, "GreaterThanEqual", "valueNumber", 1, "Or")
where_filter = filter.get_filter(operator="Or")


query = "We were hungry"
query_vector = model.encode(query)
nearVector = {
"vector": query_vector
        }


field_options = ["gender", "window_text", "birth_country", "dlf", "building"]

result = client.query.get(
        "Window", field_options
    ).with_near_vector(
        nearVector
    ).with_limit(2).with_additional(['certainty']
    ).with_where(where_filter).do()
for r in result["data"]["Get"]["Window"]:
    print(r)
    print()

{'_additional': {'certainty': 0.8350925743579865}, 'birth_country': 'Germany', 'building': 2, 'dlf': 3, 'gender': 'M', 'window_text': 'I was so hungry. I was always so hungry. So I went up there.'}

{'_additional': {'certainty': 0.8327587842941284}, 'birth_country': 'Belgium', 'building': 2, 'dlf': 0, 'gender': 'F', 'window_text': 'I ate. I... uh...I...uh...we had...uh....'}

